In [1]:
# Import required modules
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
# These are the course broad categories with directory links on udemy
broadCategory = {
    "development":"https://www.udemy.com/courses/development/",
    "business":"https://www.udemy.com/courses/business/",
    "financeAndAccounting":"https://www.udemy.com/courses/finance-and-accounting/",
    "itAndSoftware":"https://www.udemy.com/courses/it-and-software/",
    "officeProductivity":"https://www.udemy.com/courses/office-productivity/",
    "personalDevelopment":"https://www.udemy.com/courses/personal-development/",
    "design":"https://www.udemy.com/courses/design/",
    "marketing":"https://www.udemy.com/courses/marketing/",
    "lifeStyle":"https://www.udemy.com/courses/lifestyle/",
    "photographyAndVideo":"https://www.udemy.com/courses/photography-and-video/",
    "healthAndFitness":"https://www.udemy.com/courses/health-and-fitness/",
    "teachingAndAcademics":"https://www.udemy.com/courses/teaching-and-academics/",
    "music":"https://www.udemy.com/courses/music/"
}

# Create a df with category name and corresponding links
catDf = pd.DataFrame(broadCategory.items(), columns=["broadCat", "link"]).set_index("broadCat")

In [3]:
# This function scrapes individual course links from a category
def scrapeCourseLink(url, start_page, end_page):
    """url = broadcategory link,
    start_page = page the scraping will be started from,
    end_page = page the scraping will be ended on,
    return = individual course links"""
    
    # Initiate web driver
    driver = webdriver.Chrome(r"/home/faysal/Documents/utilities/chromedriver")
    driver.get(url[0])
    time.sleep(5)
    
    # Get total no of page
    totalPage = driver.find_elements_by_css_selector(".udlite-heading-sm.pagination--page--1H0A2")[-1].text
    
    # Generate cover pages for broad category
    coverPage = []
    for pg in range(1, int(totalPage)+1):
        coverPage.append(url[0] + f"?p={pg}")
    
    # Scrapes all the course links for a broad category
    courseLink = []
    for lnk in coverPage[start_page:end_page]:
        driver.get(lnk)
        time.sleep(6)
        allLinks = driver.find_elements_by_css_selector("div.popper--popper--2r2To.popper--popper-hover--3YydE a")
        for lnk in allLinks:
            courseLink.append(lnk.get_attribute("href"))
            
    # Filters out non-course links
    courseLink = list(filter(lambda x: "/course/" in x, courseLink))
    
    # Drop duplicate course links
    courseLink = list(set(courseLink))
    df = pd.DataFrame({
        "courseLink":courseLink
    })
    
    # Insert category name
    df["category"] = url.name
    driver.close()
    return df.to_excel(f"{url.name}_courseLink.xlsx", index=None)

In [4]:
# Lets scrape course links for office productivity category for first 20 page and save the output file
scrapeCourseLink(catDf.loc["officeProductivity"], 0, 20)